# COGS 188 - Project Proposal

# Project Description

You have the choice of doing either (1) an AI solve a problem style project or (2) run a Special Topics class on a topic of your choice.  If you want to do (2) you should fill out the _other_ proposal for that. This is the proposal description for (1).

You will design and execute a machine learning project. There are a few constraints on the nature of the allowed project. 
- The problem addressed will not be a "toy problem" or "common training students problem" like 8-Queens or a small Traveling Salesman Problem or similar
- If its the kind of problem (e.g., RL) that interacts with a simulator or live task, then the problem will have a reasonably complex action space. For instance, a wupus world kind of thing with a 9x9 grid is definitely too small.  A simulated mountain car with a less complex 2-d road and simplified dynamics seems like a fairly low achievement level.  A more complex 3-d mountain car simulation with large extent and realistic dynamics, sure sounds great!
- If its the kind of problem that uses a dataset, then the dataset will have >1k observations and >5 variables. I'd prefer more like >10k observations and >10 variables. A general rule is that if you have >100x more observations than variables, your solution will likely generalize a lot better. The goal of training an unsupervised machine learning model is to learn the underlying pattern in a dataset in order to generalize well to unseen data, so choosing a large dataset is very important.
- The project must include some elements we talked about in the course
- The project will include a model selection and/or feature selection component where you will be looking for the best setup to maximize the performance of your AI system. Generally RL tasks may require a huge amount of training, so extensive grid search is unlikely to be possible. However expoloring a few reasonable hyper-parameters may still be possible. 
- You will evaluate the performance of your AI system using more than one appropriate metric
- You will be writing a report describing and discussing these accomplishments


Feel free to delete this description section when you hand in your proposal.

# Names

Hopefully your team is at least this good. Obviously you should replace these with your names.

- Ashesh Kaji
- Abhay Anand
- Varun Pillai
- Ansh Bhatnagar

# Abstract 
The goal of this project is to train a Reinforcement Learning (RL) Classifier on autonomous vehicles. We plan to use DonkeyCar Simulator to navigate the "Warren Feild" track and collect our data. The data we plan to collect includes visual input from the front-facing camera, speed and steering angle change. These will be collected during training in the simulated environment. We implement and compare two deep RL algorithms: Actor-Critic and Proximal Policy Optimization (PPO), both designed for continuous action spaces. Moreover we plan to include a simple Q learning model to show how inefficient/ ineffective it is for more complex problems. We will use the gathered data to train agents to take optimal actions such as steering left/right, acceleration, and braking based on the cars current position. Performance will be evaluated using key metrics such as cumulative reward, lap completion time, and DonkeySim's own accuracy rating. By comparing these metrics across different models and training scenarios, we aim to determine which RL method provides the most robust and efficient control for autonomous driving in simulated environments.

# Background

Fill in the background and discuss the kind of prior work that has gone on in this research area here. **Use inline citation** to specify which references support which statements.  You can do that through HTML footnotes (demonstrated here). I used to reccommend Markdown footnotes (google is your friend) because they are simpler but recently I have had some problems with them working for me whereas HTML ones always work so far. So use the method that works for you, but do use inline citations.

Here is an example of inline citation. After government genocide in the 20th century, real birds were replaced with surveillance drones designed to look just like birds<a name="lorenz"></a>[<sup>[1]</sup>](#lorenznote). Use a minimum of 3 to 5 citations, but we prefer more <a name="admonish"></a>[<sup>[2]</sup>](#admonishnote). You need enough citations to fully explain and back up important facts. 

Remeber you are trying to explain why someone would want to answer your question or why your hypothesis is in the form that you've stated. 

# Problem Statement

Clearly describe the problem that you are solving. Avoid ambiguous words. The problem described should be well defined and should have at least one ML-relevant potential solution. Additionally, describe the problem thoroughly such that it is clear that the problem is quantifiable (the problem can be expressed in mathematical or logical terms), measurable (the problem can be measured by some metric and clearly observed), and replicable (the problem can be reproduced and occurs more than once).

# Data
## Original Instructions
You should have a strong idea of what dataset(s) will be used to accomplish this project. 

If you know what (some) of the data you will use, please give the following information for each dataset:
- link/reference to obtain it
- description of the size of the dataset (# of variables, # of observations)
- what an observation consists of
- what some critical variables are, how they are represented
- any special handling, transformations, cleaning, etc will be needed

If you don't yet know what your dataset(s) will be, you should describe what you desire in terms of the above bullets.

## Data
We will be using the CARLA open-source gym environment for our project. The CARLA simulator is a platform for autonomous driving research, providing a realistic urban driving environment with various weather conditions, traffic scenarios, and sensor modalities.

# Proposed Solution
## original instructions
In this section, clearly describe a solution to the problem. The solution should be applicable to the project domain and appropriate for the dataset(s) or input(s) given. Provide enough detail (e.g., algorithmic description and/or theoretical properties) to convince us that your solution is applicable. Why might your solution work? Make sure to describe how the solution will be tested.  

If you know details already, describe how (e.g., library used, function calls) you plan to implement the solution in a way that is reproducible.

If it is appropriate to the problem statement, describe a benchmark model<a name="sota"></a>[<sup>[3]</sup>](#sotanote) against which your solution will be compared. 

## Proposed Solution
We will implement a deep reinforcement learning (RL) approach to train an agent to navigate the CARLA environment. The RL agent will learn to make decisions based on the state of the environment, which includes visual input from the front-facing camera, speed, and steering angle. We will implement three algprithms, one monte carlo based, and two deep learning based:
1. **Monte Carlo Q-Learning**: A simple Q-learning algorithm that uses a table to store the Q-values for each state-action pair. This approach will serve as a baseline to compare the performance of more complex algorithms.
2. **Actor-Critic**: A deep RL algorithm that uses two neural networks: an actor network to select actions and a critic network to evaluate the actions taken. The actor network will be trained to maximize the expected cumulative reward, while the critic network will be trained to minimize the difference between the predicted and actual rewards.
3. **Proximal Policy Optimization (PPO)**: A more advanced deep RL algorithm that uses a surrogate objective function to optimize the policy. PPO is known for its stability and sample efficiency, making it suitable for continuous action spaces like autonomous driving.


## Porposed Code (has not been run, unsure if it will work)

```python
import pytorch_lightning as pl
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn import functional as F
from environment import CarlaEnv
import random

def preprocess(state):
    state_tensor = torch.tensor(state, dtype=torch.float32).permute(2, 0, 1) / 255.0
    return state_tensor

class PolicyNetwork(nn.Module):
    def __init__(self, num_actions):
        super(PolicyNetwork, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc = nn.Sequential(
            nn.Linear(7 * 7 * 64, 512),
            nn.ReLU()
        )
        self.actor = nn.Linear(512, num_actions)
        self.critic = nn.Linear(512, 1)

    def forward(self, x):
        features = self.cnn(x)
        features = self.fc(features)
        logits = self.actor(features)
        value = self.critic(features)
        return logits, value

class PPOAgent(pl.LightningModule):
    def __init__(
        self,
        env: CarlaEnv,
        lr: float = 3e-4,
        gamma: float = 0.99,
        lam: float = 0.95,
        clip_epsilon: float = 0.2,
        rollout_steps: int = 2048,
    ):
        super(PPOAgent, self).__init__()
        self.env = env
        self.lr = lr
        self.gamma = gamma
        self.lam = lam
        self.clip_epsilon = clip_epsilon
        self.rollout_steps = rollout_steps
        self.num_actions = self.env.action_space.n

        self.policy_net = PolicyNetwork(self.num_actions)
        self.save_hyperparameters(ignore=["env"])

    def forward(self, x):
        return self.policy_net(x)

    def collect_rollout(self):
        states, actions, rewards, dones, log_probs, values = [], [], [], [], [], []
        state = self.env.reset()
        for _ in range(self.rollout_steps):
            state_tensor = preprocess(state).to(self.device)
            logits, value = self.policy_net(state_tensor.unsqueeze(0))
            probs = F.softmax(logits, dim=-1)
            action = torch.multinomial(probs, num_samples=1)
            log_prob = torch.log(probs.gather(1, action) + 1e-8)
            
            next_state, reward, done, _ = self.env.step(action.item())

            states.append(state_tensor.unsqueeze(0))
            actions.append(action)
            rewards.append(torch.tensor([reward], dtype=torch.float, device=self.device))
            dones.append(torch.tensor([float(done)], dtype=torch.float, device=self.device))
            log_probs.append(log_prob)
            values.append(value)

            state = next_state
            if done:
                state = self.env.reset()

        rollout = {
            'states': torch.cat(states, dim=0),
            'actions': torch.cat(actions, dim=0),
            'rewards': torch.cat(rewards, dim=0),
            'dones': torch.cat(dones, dim=0),
            'log_probs': torch.cat(log_probs, dim=0),
            'values': torch.cat(values, dim=0)
        }
        return rollout

    def compute_gae(self, rewards, values, dones):
        advantages = []
        gae = 0
        values = values.squeeze(-1)
        for step in reversed(range(len(rewards))):
            next_value = values[step + 1] if step < len(rewards) - 1 else 0
            delta = rewards[step] + self.gamma * (1 - dones[step]) * next_value - values[step]
            gae = delta + self.gamma * self.lam * (1 - dones[step]) * gae
            advantages.insert(0, gae)
        advantages = torch.stack(advantages)
        returns = advantages + values
        return advantages, returns

    def training_step(self, batch, batch_idx):
        rollout = self.collect_rollout()
        advantages, returns = self.compute_gae(
            rollout['rewards'], rollout['values'], rollout['dones']
        )
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

        logits, values_new = self.policy_net(rollout['states'])
        probs = F.softmax(logits, dim=-1)
        new_log_probs = torch.log(probs.gather(1, rollout['actions']) + 1e-8)
        ratio = torch.exp(new_log_probs - rollout['log_probs'])

        advantages = advantages.unsqueeze(1)
        surr1 = ratio * advantages
        surr2 = torch.clamp(ratio, 1.0 - self.clip_epsilon, 1.0 + self.clip_epsilon) * advantages
        policy_loss = -torch.min(surr1, surr2).mean()

        returns = returns.detach()
        value_loss = F.mse_loss(values_new.squeeze(), returns)
        loss = policy_loss + 0.5 * value_loss

        self.log("policy_loss", policy_loss, prog_bar=True)
        self.log("value_loss", value_loss, prog_bar=True)
        self.log("total_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.policy_net.parameters(), lr=self.lr)
        return optimizer

if __name__ == "__main__":
    env = CarlaEnv()
    model = PPOAgent(env)

    trainer = pl.Trainer(
        max_epochs=100,
        gpus=1 if torch.cuda.is_available() else 0,
        log_every_n_steps=10,
        checkpoint_callback=True,
    )
    trainer.fit(model, train_dataloader=[None])
    
    


# DQN implementation approach
def preprocess(state):
    state_tensor = torch.tensor(state, dtype=torch.float32).permute(2, 0, 1) / 255.0
    return state_tensor

class QNetwork(nn.Module):
    def __init__(self, num_actions):
        super(QNetwork, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1),
            nn.ReLU(),
            nn.Flatten()
        )
        self.fc = nn.Sequential(
            nn.Linear(7 * 7 * 64, 512),
            nn.ReLU()
        )
        self.out = nn.Linear(512, num_actions)

    def forward(self, x):
        features = self.cnn(x)
        features = self.fc(features)
        q_values = self.out(features)
        return q_values

class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = deque(maxlen=capacity)
    
    def push(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        batch = random.sample(self.buffer, batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        return (
            torch.cat(states),
            torch.tensor(actions, dtype=torch.long).unsqueeze(1),
            torch.tensor(rewards, dtype=torch.float32).unsqueeze(1),
            torch.cat(next_states),
            torch.tensor(dones, dtype=torch.float32).unsqueeze(1)
        )
    
    def __len__(self):
        return len(self.buffer)

class DQNAgent(pl.LightningModule):
    def __init__(
        self,
        env: CarlaEnv,
        lr: float = 1e-3,
        gamma: float = 0.99,
        buffer_capacity: int = 10000,
        batch_size: int = 32,
        epsilon_start: float = 1.0,
        epsilon_end: float = 0.1,
        epsilon_decay: int = 10000,
        target_update_freq: int = 1000
    ):
        super(DQNAgent, self).__init__()
        self.env = env
        self.lr = lr
        self.gamma = gamma
        self.batch_size = batch_size
        self.epsilon_start = epsilon_start
        self.epsilon_end = epsilon_end
        self.epsilon_decay = epsilon_decay
        self.target_update_freq = target_update_freq

        self.num_actions = self.env.action_space.n
        self.online_net = QNetwork(self.num_actions)
        self.target_net = QNetwork(self.num_actions)
        self.target_net.load_state_dict(self.online_net.state_dict())
        self.replay_buffer = ReplayBuffer(buffer_capacity)
        self.total_steps = 0

        self.save_hyperparameters(ignore=["env"])

    def forward(self, x):
        return self.online_net(x)

    def epsilon_by_step(self, step):
        return self.epsilon_end + (self.epsilon_start - self.epsilon_end) * np.exp(-step / self.epsilon_decay)

    def training_step(self, batch, batch_idx):
        if not hasattr(self.env, "current_state"):
            state = self.env.reset()
            self.env.current_state = state
        else:
            state = self.env.current_state

        state_tensor = preprocess(state).to(self.device).unsqueeze(0)
        epsilon = self.epsilon_by_step(self.total_steps)
        
        if random.random() < epsilon:
            action = random.randrange(self.num_actions)
        else:
            with torch.no_grad():
                q_values = self.online_net(state_tensor)
                action = q_values.argmax(dim=1).item()
        
        next_state, reward, done, _ = self.env.step(action)
        next_state_tensor = preprocess(next_state).to(self.device).unsqueeze(0)
        self.replay_buffer.push(state_tensor, action, reward, next_state_tensor, done)

        if done:
            state = self.env.reset()
        else:
            state = next_state
        self.env.current_state = state
        self.total_steps += 1

        if self.total_steps % self.target_update_freq == 0:
            self.target_net.load_state_dict(self.online_net.state_dict())

        if len(self.replay_buffer) < self.batch_size:
            return None

        states, actions, rewards, next_states, dones = self.replay_buffer.sample(self.batch_size)
        states = states.to(self.device)
        actions = actions.to(self.device)
        rewards = rewards.to(self.device)
        next_states = next_states.to(self.device)
        dones = dones.to(self.device)

        current_q = self.online_net(states).gather(1, actions)
        with torch.no_grad():
            max_next_q = self.target_net(next_states).max(dim=1, keepdim=True)[0]
            target_q = rewards + self.gamma * max_next_q * (1 - dones)
        
        loss = F.mse_loss(current_q, target_q)
        self.log("loss", loss, prog_bar=True)
        self.log("epsilon", epsilon, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.online_net.parameters(), lr=self.lr)
        return optimizer

if __name__ == "__main__":
    env = CarlaEnv()
    model = DQNAgent(env)

    trainer = pl.Trainer(
        max_epochs=100,
        gpus=1 if torch.cuda.is_available() else 0,
        log_every_n_steps=10,
        checkpoint_callback=True,
    )
    trainer.fit(model, train_dataloader=[None])
```

# Evaluation Metrics

Propose at least one evaluation metric that can be used to quantify the performance of both the benchmark model and the solution model. The evaluation metric(s) you propose should be appropriate given the context of the data, the problem statement, and the intended solution. Describe how the evaluation metric(s) are derived and provide an example of their mathematical representations (if applicable). Complex evaluation metrics should be clearly defined and quantifiable (can be expressed in mathematical or logical terms).

# Ethics & Privacy

If your project has obvious potential concerns with ethics or data privacy discuss that here.  Almost every ML project put into production can have ethical implications if you use your imagination. Use your imagination. Get creative!

Even if you can't come up with an obvious ethical concern that should be addressed, you should know that a large number of ML projects that go into producation have unintended consequences and ethical problems once in production. How will your team address these issues?

Consider a tool to help you address the potential issues such as https://deon.drivendata.org

# Team Expectations 

Put things here that cement how you will interact/communicate as a team, how you will handle conflict and difficulty, how you will handle making decisions and setting goals/schedule, how much work you expect from each other, how you will handle deadlines, etc...
* Regular updates via WhatsApp, weekly check-ins, quick status updates when problems/issues arise 
* Set realistic deadlines, and communicate on time if a problem comes up
* Decision-making should be done collaboratively, discussions for major decisions
* Address conflicts respectfully

# Project Timeline Proposal

Replace this with something meaningful that is appropriate for your needs. It doesn't have to be something that fits this format.  It doesn't have to be set in stone... "no battle plan survives contact with the enemy". But you need a battle plan nonetheless, and you need to keep it updated so you understand what you are trying to accomplish, who's responsible for what, and what the expected due dates are for each item.

| Meeting Date  | Meeting Time| Completed Before Meeting  | Discuss at Meeting |
|---|---|---|---|
| 1/20  |  1 PM |  Brainstorm topics/questions (all)  | Determine best form of communication; Discuss and decide on final project topic; discuss hypothesis; begin background research | 
| 1/26  |  10 AM |  Do background research on topic (Pelé) | Discuss ideal dataset(s) and ethics; draft project proposal | 
| 2/1  | 10 AM  | Edit, finalize, and submit proposal; Search for datasets (Beckenbaur)  | Discuss Wrangling and possible analytical approaches; Assign group members to lead each specific part   |
| 2/14  | 6 PM  | Import & Wrangle Data ,do some EDA (Maradonna) | Review/Edit wrangling/EDA; Discuss Analysis Plan   |
| 2/23  | 12 PM  | Finalize wrangling/EDA; Begin programming for project (Cruyff) | Discuss/edit project code; Complete project |
| 3/13  | 12 PM  | Complete analysis; Draft results/conclusion/discussion (Carlos)| Discuss/edit full project |
| 3/19  | Before 11:59 PM  | NA | Turn in Final Project  |

# Footnotes
<a name="lorenznote"></a>1.[^](#lorenz): Lorenz, T. (9 Dec 2021) Birds Aren’t Real, or Are They? Inside a Gen Z Conspiracy Theory. *The New York Times*. https://www.nytimes.com/2021/12/09/technology/birds-arent-real-gen-z-misinformation.html<br> 
<a name="admonishnote"></a>2.[^](#admonish): Also refs should be important to the background, not some randomly chosen vaguely related stuff. Include a web link if possible in refs as above.<br>
<a name="sotanote"></a>3.[^](#sota): Perhaps the current state of the art solution such as you see on [Papers with code](https://paperswithcode.com/sota). Or maybe not SOTA, but rather a standard textbook/Kaggle solution to this kind of problem
